In [3]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.5 MB/s eta 0:00:00


In [5]:
# Import the necessary libraries

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from category_encoders import OneHotEncoder

# Load the dataset
url = "https://raw.githubusercontent.com/BossyNine/bakery_sales_prediction/main/sourcedata/cleaned_data/6_weather_parameters.csv"
df = pd.read_csv(url)

# Read the file and check it out
# df = pd.read_csv(r'/Users/jberndt/Desktop/2-2-opencampus/X-opencampus_ML/2_DS_and_ML/6-homework_week06/6_weather_parameters.csv')
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999 entries, 0 to 6998
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Datum                6999 non-null   object 
 1   Bewoelkung           6999 non-null   float64
 2   Temperatur           6999 non-null   float64
 3   Windgeschwindigkeit  6999 non-null   float64
 4   Wettercode           6999 non-null   float64
 5   KielerWoche          6999 non-null   float64
 6   Warengruppe          6999 non-null   float64
 7   Umsatz               6999 non-null   float64
 8   Feiertage            6999 non-null   float64
 9   Ferientage           6999 non-null   float64
 10  Wetterklasse         6999 non-null   int64  
 11  Niederschlag         6999 non-null   float64
 12  year                 6999 non-null   int64  
 13  month                6999 non-null   int64  
 14  week                 6999 non-null   int64  
 15  weekday              6999 non-null   i

,Datum,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KielerWoche,Warengruppe,Umsatz,Feiertage,Ferientage,Wetterklasse,Niederschlag,year,month,week,weekday,day_month,season_str,season
0,2013-07-01,6.0,17.8375,15.0,20.0,0.0,1.0,148.828353,0.0,1.0,6,0.3,2013,7,27,0,1,summer,2
1,2013-07-01,6.0,17.8375,15.0,20.0,0.0,2.0,535.856285,0.0,1.0,6,0.3,2013,7,27,0,1,summer,2
2,2013-07-01,6.0,17.8375,15.0,20.0,0.0,3.0,201.198426,0.0,1.0,6,0.3,2013,7,27,0,1,summer,2
3,2013-07-01,6.0,17.8375,15.0,20.0,0.0,4.0,65.890169,0.0,1.0,6,0.3,2013,7,27,0,1,summer,2
4,2013-07-01,6.0,17.8375,15.0,20.0,0.0,5.0,317.475875,0.0,1.0,6,0.3,2013,7,27,0,1,summer,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6994,2018-07-30,6.0,27.7875,10.0,21.0,0.0,1.0,148.541340,0.0,1.0,6,0.0,2018,7,31,0,30,summer,2
6995,2018-07-30,6.0,27.7875,10.0,21.0,0.0,2.0,616.358562,0.0,1.0,6,0.0,2018,7,31,0,30,summer,2
6996,2018-07-30,6.0,27.7875,10.0,21.0,0.0,3.0,348.770346,0.0,1.0,6,0.0,2018,7,31,0,30,summer,2
6997,2018-07-30,6.0,27.7875,10.0,21.0,0.0,4.0,71.822977,0.0,1.0,6,0.0,2018,7,31,0,30,summer,2


In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

# Selecting the desired columns including 'Warengruppe'
features = ['Temperatur', 'KielerWoche', 'Warengruppe', 'Bewoelkung', 'Windgeschwindigkeit', 'Feiertage', 'Ferientage', 'Wetterklasse']
X = df[features]
y = df['Umsatz']

# Checking for missing values and filling if necessary
if X.isnull().sum().sum() > 0 or y.isnull().sum() > 0:
    X = X.fillna(X.mean())
    y = y.fillna(y.mean())

# Standardizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Adding polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

# Random Forest Regressor with cross-validation
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
cv_scores = cross_val_score(rf_regressor, X_poly, y, cv=5, scoring='r2')

# Train-test split for detailed evaluation
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

rf_regressor.fit(X_train, y_train)
y_pred = rf_regressor.predict(X_test)

# Evaluation
results = {
    "Cross-validated R2 scores": cv_scores,
    "Mean R2 score": np.mean(cv_scores),
    "R2 score": r2_score(y_test, y_pred),
    "Mean squared error": mean_squared_error(y_test, y_pred),
    "Mean absolute error": mean_absolute_error(y_test, y_pred)
}

results


{'Cross-validated R2 scores': array([0.72510205, 0.68681934, 0.64986923, 0.58695411, 0.59564355]),
 'Mean R2 score': 0.6488776541555261,
 'R2 score': 0.6875782644249097,
 'Mean squared error': 6195.638521287795,
 'Mean absolute error': 45.99159463170172}

The updated model using the Random Forest Regressor and incorporating polynomial features has yielded the following results:

Cross-validated R2 scores: [0.72510205, 0.68681934, 0.64986923, 0.58695411, 0.59564355]  
Mean R2 score: 0.6488776541555261  
R2 score on test set: 0.6875782644249097  
Mean squared error (MSE): 6195.638521287795  
Mean absolute error (MAE): 45.99159463170172

The R2 score has improved substantially, indicating that the model now explains a larger portion of the variance in the target variable Umsatz.
The MSE and MAE have both decreased, indicating that the prediction errors are smaller and more reasonable.
This suggests that including Warengruppe as a feature and using a Random Forest Regressor with polynomial features was a beneficial approach.

In [9]:
# Remove features 'Feiertage' and 'Ferientage' from the solution to check impact

# Selecting the desired columns excluding 'Feiertage' and 'Ferientage'
features = ['Temperatur', 'KielerWoche', 'Warengruppe', 'Bewoelkung', 'Windgeschwindigkeit', 'Wetterklasse']
X = df[features]
y = df['Umsatz']

# Checking for missing values and filling if necessary
if X.isnull().sum().sum() > 0 or y.isnull().sum() > 0:
    X = X.fillna(X.mean())
    y = y.fillna(y.mean())

# Standardizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Adding polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

# Random Forest Regressor with cross-validation
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
cv_scores = cross_val_score(rf_regressor, X_poly, y, cv=5, scoring='r2')

# Train-test split for detailed evaluation
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

rf_regressor.fit(X_train, y_train)
y_pred = rf_regressor.predict(X_test)

# Evaluation
results = {
    "Cross-validated R2 scores": cv_scores,
    "Mean R2 score": np.mean(cv_scores),
    "R2 score": r2_score(y_test, y_pred),
    "Mean squared error": mean_squared_error(y_test, y_pred),
    "Mean absolute error": mean_absolute_error(y_test, y_pred)
}

results


{'Cross-validated R2 scores': array([0.70979694, 0.6863177 , 0.62779108, 0.58657693, 0.62508494]),
 'Mean R2 score': 0.6471135178198661,
 'R2 score': 0.6707846997579459,
 'Mean squared error': 6528.671867923726,
 'Mean absolute error': 47.63266441025559}

Comparison:  
The Mean R2 score is slightly lower compared to when 'Feiertage' and 'Ferientage' were included (0.647 vs. 0.649).  
The R2 score on the test set has decreased slightly (0.671 vs. 0.688).  
The MSE has increased slightly (6528.67 vs. 6195.64).  
The MAE has also increased slightly (47.63 vs. 45.99).  
Conclusion:  
Removing 'Feiertage' and 'Ferientage' has resulted in a minor decrease in model performance. These features appear to provide some useful information for predicting 'Umsatz'. Hence, it might be beneficial to keep 'Feiertage' and 'Ferientage' in the model.

In [10]:
# Selecting the desired columns excluding 'KielerWoche'
features = ['Temperatur', 'Warengruppe', 'Bewoelkung', 'Windgeschwindigkeit', 'Feiertage', 'Ferientage', 'Wetterklasse']
X = df[features]
y = df['Umsatz']

# Checking for missing values and filling if necessary
if X.isnull().sum().sum() > 0 or y.isnull().sum() > 0:
    X = X.fillna(X.mean())
    y = y.fillna(y.mean())

# Standardizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Adding polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

# Random Forest Regressor with cross-validation
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
cv_scores = cross_val_score(rf_regressor, X_poly, y, cv=5, scoring='r2')

# Train-test split for detailed evaluation
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

rf_regressor.fit(X_train, y_train)
y_pred = rf_regressor.predict(X_test)

# Evaluation
results = {
    "Cross-validated R2 scores": cv_scores,
    "Mean R2 score": np.mean(cv_scores),
    "R2 score": r2_score(y_test, y_pred),
    "Mean squared error": mean_squared_error(y_test, y_pred),
    "Mean absolute error": mean_absolute_error(y_test, y_pred)
}

results


{'Cross-validated R2 scores': array([0.71979943, 0.68904311, 0.63769002, 0.57488845, 0.59155483]),
 'Mean R2 score': 0.642595168265497,
 'R2 score': 0.6876077612017122,
 'Mean squared error': 6195.053570415939,
 'Mean absolute error': 46.32903316781526}

Comparison:  
The Mean R2 score is slightly lower compared to when 'KielerWoche' was included (0.643 vs. 0.649).  
The R2 score on the test set is almost the same (0.688 vs. 0.688).  
The MSE is very close (6195.05 vs. 6195.64).  
The MAE is slightly higher (46.33 vs. 45.99).  
Conclusion:  
Removing 'KielerWoche' did not significantly impact the model's performance. The slight differences in the metrics indicate that 'KielerWoche' may not be a crucial feature for predicting 'Umsatz'.

Given these results, you could choose to exclude 'KielerWoche' without losing significant predictive power.

In [11]:
# Selecting the desired columns excluding 'Temperatur'
features = ['KielerWoche', 'Warengruppe', 'Bewoelkung', 'Windgeschwindigkeit', 'Feiertage', 'Ferientage', 'Wetterklasse']
X = df[features]
y = df['Umsatz']

# Checking for missing values and filling if necessary
if X.isnull().sum().sum() > 0 or y.isnull().sum() > 0:
    X = X.fillna(X.mean())
    y = y.fillna(y.mean())

# Standardizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Adding polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

# Random Forest Regressor with cross-validation
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
cv_scores = cross_val_score(rf_regressor, X_poly, y, cv=5, scoring='r2')

# Train-test split for detailed evaluation
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

rf_regressor.fit(X_train, y_train)
y_pred = rf_regressor.predict(X_test)

# Evaluation
results = {
    "Cross-validated R2 scores": cv_scores,
    "Mean R2 score": np.mean(cv_scores),
    "R2 score": r2_score(y_test, y_pred),
    "Mean squared error": mean_squared_error(y_test, y_pred),
    "Mean absolute error": mean_absolute_error(y_test, y_pred)
}

results


{'Cross-validated R2 scores': array([0.58161765, 0.62938234, 0.45610775, 0.50656111, 0.55825951]),
 'Mean R2 score': 0.5463856717616978,
 'R2 score': 0.5885306865864346,
 'Mean squared error': 8159.852014842291,
 'Mean absolute error': 54.43894877711045}

Comparison:  
The Mean R2 score is significantly lower compared to when 'Temperatur' was included (0.546 vs. 0.649).  
The R2 score on the test set has decreased notably (0.589 vs. 0.688).  
The MSE has increased (8159.85 vs. 6195.64).  
The MAE has also increased (54.44 vs. 45.99).  
Conclusion:  
Removing 'Temperatur' has led to a significant decrease in model performance, indicating that 'Temperatur' is an important feature for predicting 'Umsatz'.   
It is advisable to keep 'Temperatur' in the model to maintain better predictive accuracy!

In [12]:
# Selecting the desired columns excluding 'Wetterklasse'
features = ['Temperatur', 'KielerWoche', 'Warengruppe', 'Bewoelkung', 'Windgeschwindigkeit', 'Feiertage', 'Ferientage']
X = df[features]
y = df['Umsatz']

# Checking for missing values and filling if necessary
if X.isnull().sum().sum() > 0 or y.isnull().sum() > 0:
    X = X.fillna(X.mean())
    y = y.fillna(y.mean())

# Standardizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Adding polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

# Random Forest Regressor with cross-validation
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
cv_scores = cross_val_score(rf_regressor, X_poly, y, cv=5, scoring='r2')

# Train-test split for detailed evaluation
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

rf_regressor.fit(X_train, y_train)
y_pred = rf_regressor.predict(X_test)

# Evaluation
results = {
    "Cross-validated R2 scores": cv_scores,
    "Mean R2 score": np.mean(cv_scores),
    "R2 score": r2_score(y_test, y_pred),
    "Mean squared error": mean_squared_error(y_test, y_pred),
    "Mean absolute error": mean_absolute_error(y_test, y_pred)
}

results


{'Cross-validated R2 scores': array([0.71561174, 0.67814811, 0.65156943, 0.56171709, 0.58514374]),
 'Mean R2 score': 0.6384380207737101,
 'R2 score': 0.6670723477967039,
 'Mean squared error': 6602.291556302038,
 'Mean absolute error': 46.90788054038095}

Comparison:  
The Mean R2 score is slightly lower compared to when 'Wetterklasse' was included (0.638 vs. 0.649).  
The R2 score on the test set has decreased slightly (0.667 vs. 0.688).  
The MSE has increased (6602.29 vs. 6195.64).  
The MAE is slightly higher (46.91 vs. 45.99).  
Conclusion:  
Removing 'Wetterklasse' has led to a slight decrease in model performance, indicating that 'Wetterklasse' contributes to the predictive power of the model.

Based on these observations, it would be advisable to keep 'Wetterklasse' in the model to maintain better predictive accuracy.

In [13]:
# Selecting the desired columns excluding 'Bewoelkung'
features = ['Temperatur', 'KielerWoche', 'Warengruppe', 'Windgeschwindigkeit', 'Feiertage', 'Ferientage', 'Wetterklasse']
X = df[features]
y = df['Umsatz']

# Checking for missing values and filling if necessary
if X.isnull().sum().sum() > 0 or y.isnull().sum() > 0:
    X = X.fillna(X.mean())
    y = y.fillna(y.mean())

# Standardizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Adding polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

# Random Forest Regressor with cross-validation
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
cv_scores = cross_val_score(rf_regressor, X_poly, y, cv=5, scoring='r2')

# Train-test split for detailed evaluation
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

rf_regressor.fit(X_train, y_train)
y_pred = rf_regressor.predict(X_test)

# Evaluation
results = {
    "Cross-validated R2 scores": cv_scores,
    "Mean R2 score": np.mean(cv_scores),
    "R2 score": r2_score(y_test, y_pred),
    "Mean squared error": mean_squared_error(y_test, y_pred),
    "Mean absolute error": mean_absolute_error(y_test, y_pred)
}

results


{'Cross-validated R2 scores': array([0.71880587, 0.68058319, 0.63986169, 0.59933543, 0.593011  ]),
 'Mean R2 score': 0.6463194348548796,
 'R2 score': 0.6820063893808962,
 'Mean squared error': 6306.134430271037,
 'Mean absolute error': 46.128360643518974}

Comparison:  
The Mean R2 score is slightly lower compared to when 'Bewoelkung' was included (0.646 vs. 0.649).  
The R2 score on the test set has decreased slightly (0.682 vs. 0.688).  
The MSE has increased (6306.13 vs. 6195.64).  
The MAE is slightly higher (46.13 vs. 45.99).  
Conclusion:  
Removing 'Bewoelkung' has led to a slight decrease in model performance, indicating that 'Bewoelkung' contributes to the predictive power of the model, although not significantly.

Based on these observations, it would be advisable to keep 'Bewoelkung' in the model to maintain better predictive accuracy.

In [14]:
# Selecting the desired columns excluding 'Windgeschwindigkeit'
features = ['Temperatur', 'KielerWoche', 'Warengruppe', 'Bewoelkung', 'Feiertage', 'Ferientage', 'Wetterklasse']
X = df[features]
y = df['Umsatz']

# Checking for missing values and filling if necessary
if X.isnull().sum().sum() > 0 or y.isnull().sum() > 0:
    X = X.fillna(X.mean())
    y = y.fillna(y.mean())

# Standardizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Adding polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

# Random Forest Regressor with cross-validation
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
cv_scores = cross_val_score(rf_regressor, X_poly, y, cv=5, scoring='r2')

# Train-test split for detailed evaluation
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

rf_regressor.fit(X_train, y_train)
y_pred = rf_regressor.predict(X_test)

# Evaluation
results = {
    "Cross-validated R2 scores": cv_scores,
    "Mean R2 score": np.mean(cv_scores),
    "R2 score": r2_score(y_test, y_pred),
    "Mean squared error": mean_squared_error(y_test, y_pred),
    "Mean absolute error": mean_absolute_error(y_test, y_pred)
}

results


{'Cross-validated R2 scores': array([0.71177809, 0.66154863, 0.6309924 , 0.55741207, 0.53634127]),
 'Mean R2 score': 0.6196144905259606,
 'R2 score': 0.674763321923051,
 'Mean squared error': 6449.771772504949,
 'Mean absolute error': 47.360921179751685}

Comparison:  
The Mean R2 score is lower compared to when 'Windgeschwindigkeit' was included (0.620 vs. 0.649).  
The R2 score on the test set has decreased slightly (0.675 vs. 0.688).  
The MSE has increased (6449.77 vs. 6195.64).  
The MAE is slightly higher (47.36 vs. 45.99).  
Conclusion:  
Removing 'Windgeschwindigkeit' has led to a decrease in model performance, indicating that 'Windgeschwindigkeit' contributes to the predictive power of the model.

Based on these observations, it would be advisable to keep 'Windgeschwindigkeit' in the model to maintain better predictive accuracy.

In [15]:
# Selecting the desired columns including 'Niederschlag'
features = ['Temperatur', 'KielerWoche', 'Warengruppe', 'Bewoelkung', 'Windgeschwindigkeit', 'Feiertage', 'Ferientage', 'Wetterklasse', 'Niederschlag']
X = df[features]
y = df['Umsatz']

# Checking for missing values and filling if necessary
if X.isnull().sum().sum() > 0 or y.isnull().sum() > 0:
    X = X.fillna(X.mean())
    y = y.fillna(y.mean())

# Standardizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Adding polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

# Random Forest Regressor with cross-validation
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
cv_scores = cross_val_score(rf_regressor, X_poly, y, cv=5, scoring='r2')

# Train-test split for detailed evaluation
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

rf_regressor.fit(X_train, y_train)
y_pred = rf_regressor.predict(X_test)

# Evaluation
results = {
    "Cross-validated R2 scores": cv_scores,
    "Mean R2 score": np.mean(cv_scores),
    "R2 score": r2_score(y_test, y_pred),
    "Mean squared error": mean_squared_error(y_test, y_pred),
    "Mean absolute error": mean_absolute_error(y_test, y_pred)
}

results


{'Cross-validated R2 scores': array([0.7297873 , 0.6889417 , 0.65568813, 0.59231637, 0.61656247]),
 'Mean R2 score': 0.6566591945000697,
 'R2 score': 0.6954848698278695,
 'Mean squared error': 6038.842551516265,
 'Mean absolute error': 44.857032558501054}

Comparison:  
The Mean R2 score has slightly improved compared to the previous models (0.657 vs. 0.649).  
The R2 score on the test set is slightly higher compared to the best model we had earlier (0.695 vs. 0.688).  
The MSE is lower compared to the best model (6038.84 vs. 6195.64).
The MAE is similar to the best model (44.85 vs. 45.99).
Conclusion:
Including 'Niederschlag' has resulted in a slight improvement in the Mean R2 score, indicating that it might have some predictive power. However, the overall performance metrics such as R2 score on the test set, MSE, and MAE are very close to the best model we had without polynomial features.

Based on these observations, including 'Niederschlag' seems to add some value to the model. Therefore, it can be beneficial to include 'Niederschlag' in the feature set.

**Conclusion**:
As an *R2* score close to *0.7* is considered good, especially given the context of predictive modeling with real-world data, our model's performance metrics suggest it is reliable and effective for predicting 'Umsatz'.  
Further improvements can be made if necessary, but the current results are quite strong.
